# Transfer data

This guide shows how to transfer data from a source database into the currently connected database.

In [ ]:
# pip install 'lamindb[jupyter,bionty]'
!lamin init --storage ./test-transfer --modules bionty

In [ ]:
import lamindb as ln

ln.track("ITeOtm7bhtdq0000")

Query all artifacts in the `laminlabs/lamindata` instance and filter them to their latest versions.

In [ ]:
# query all latest artifact versions
artifacts = ln.Artifact.using("laminlabs/lamindata").filter(is_latest=True)

# convert the QuerySet to a DataFrame and show the latest 5 versions
artifacts.df().head()

You can now further subset or search the {class}`~lamindb.models.QuerySet`. Here we query by whether the description contains "tabula sapiens".

In [ ]:
artifact = artifacts.filter(description__contains="Tabula Sapiens").first()
artifact.describe()

By saving the artifact record that's currently attached to the source database instance, you transfer it to the default database instance.

In [ ]:
artifact.save()

```{dropdown} How do I know if a record is saved in the default database instance or not?

Every record has an attribute `._state.db` which can take the following values:

- `None`: the record has not yet been saved to any database
- `"default"`: the record is saved on the default database instance
- `"account/name"`: the record is saved on a non-default database instance referenced by `account/name` (e.g., `laminlabs/lamindata`)

```

The artifact record and all other feature & label records have been transferred to the current database.

In [ ]:
artifact.describe()

You see that the data itself remained in the original storage location, which has been added to the current instance's storage location as a read-only location.

In [ ]:
ln.Storage.df()

See the state of the database.

In [ ]:
ln.view()

View lineage:

In [ ]:
artifact.view_lineage()

The transferred dataset is linked to a special type of transform that stores the slug and uid of the source instance:

In [ ]:
artifact.transform.description

The transform key has shape `f"transfers/{source_instance.uid}"`:

In [ ]:
artifact.transform.key

The current notebook run is linked as the initiated_by_run of the "transfer run":

In [ ]:
artifact.run.initiated_by_run.transform

In [ ]:
# test the last 3 cells here
# TODO restore the following test
# assert artifact.transform.description == "Transfer from `laminlabs/lamindata`"
# assert artifact.transform.key == "transfers/4XIuR0tvaiXM"
# assert artifact.transform.uid == "4XIuR0tvaiXM0000"
# assert artifact.run.initiated_by_run.transform.description == "Transfer data"

# clean up test instance
!lamin delete --force test-transfer